**`機械学習　効果測定（分類）`**

①データを保存するフォルダ「data」の作成<br>
②ドライブのマウント

In [ ]:
cd data

/content/data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


CSVファイルのアップロード

In [ ]:
# pandasをインポート
import pandas as pd

In [ ]:
# digits.csvファイルを読み込んで、データフレームに変換
df = pd.read_csv('digits.csv')
# 先頭５行の表示
df.head()

,Unnamed: 0,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,...,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7,target
0,0,0,0,5,13,9,1,0,0,0,...,0,0,0,6,13,10,0,0,0,0
1,1,0,0,0,12,13,5,0,0,0,...,0,0,0,0,11,16,10,0,0,1
2,2,0,0,0,4,15,12,0,0,0,...,0,0,0,0,3,11,16,9,0,2
3,3,0,0,7,15,13,1,0,0,0,...,0,0,0,7,13,13,9,0,0,3
4,4,0,0,0,1,11,0,0,0,0,...,0,0,0,0,2,16,4,0,0,4


In [ ]:
# 欠損値の確認
df.isnull().any(axis=0)

Unnamed: 0    False
pixel_0_0     False
pixel_0_1     False
pixel_0_2     False
pixel_0_3     False
              ...  
pixel_7_4     False
pixel_7_5     False
pixel_7_6     False
pixel_7_7     False
target        False
Length: 66, dtype: bool

In [ ]:
# 欠損値がない為、まずは決定木分析で予測モデルを作成してみる
# 正解データ
t = df['target']
# 特徴量 スライス構文で取り出す
x = df.loc[ : ,'pixel_0_0':'pixel_7_7']


In [ ]:
# 訓練データとテストデータに分割　テストデータを２割
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,t,test_size=0.2,random_state = 0)
# 訓練データとテストデータの表示
print(x_train.shape)
print(x_test.shape)

(1437, 64)
(360, 64)


In [ ]:
# 決定木モデルの作成
from sklearn import tree
# とりあえず深さ２
model = tree.DecisionTreeClassifier(max_depth=2,random_state=0)

In [ ]:
# 訓練データで学習
model.fit(x_train,y_train)

# テストデータで正解率を計算
model.score(x_test,y_test)

0.2611111111111111

In [ ]:
# 深さをどれくらいにすれば正解率が高くなるか
for i in range(3,15):
  model2 = tree.DecisionTreeClassifier(max_depth=i,random_state=0)
  model2.fit(x_train,y_train)
  print("深さ",i,"の正解率",model2.score(x_test,y_test))

深さ 3 の正解率 0.425
深さ 4 の正解率 0.6277777777777778
深さ 5 の正解率 0.7305555555555555
深さ 6 の正解率 0.7916666666666666
深さ 7 の正解率 0.8333333333333334
深さ 8 の正解率 0.8444444444444444
深さ 9 の正解率 0.85
深さ 10 の正解率 0.8638888888888889
深さ 11 の正解率 0.8638888888888889
深さ 12 の正解率 0.8416666666666667
深さ 13 の正解率 0.8472222222222222
深さ 14 の正解率 0.8472222222222222


In [ ]:
# 深さを10、11が正解率が一番良い。86.38%
# 深さ11以上だと正解率が下がる。

In [ ]:
# ロジスティック回帰を使ってみる
from sklearn.linear_model import LogisticRegression

model2 = LogisticRegression(random_state= 0, C=0.1,multi_class='auto',solver='lbfgs')


In [ ]:
# 正解率の確認 96.94%
model2.fit(x_train,y_train)
model2.score(x_test,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9694444444444444

In [ ]:
# ランダムフォレストの場合 テキストP439
from sklearn.ensemble import RandomForestClassifier

model3 = RandomForestClassifier(n_estimators=200,random_state=0)

In [ ]:
# 正解率の確認 97.22%
model3.fit(x_train,y_train)
model3.score(x_test,y_test)

0.9722222222222222

In [ ]:
from pandas.core.common import random_state
# アダブーストの場合 テキストP449
from sklearn.ensemble import AdaBoostClassifier

# 最大深さ10の決定木を作る
base_model = tree.DecisionTreeClassifier(random_state=0,max_depth = 10)

# 決定木を500個作成 正解率は98.8%
model4 = AdaBoostClassifier(n_estimators=500,random_state=0,base_estimator= base_model)
model4.fit(x_train,y_train)
model4.score(x_test,y_test)

0.9888888888888889

In [ ]:
# アダブースト、作成するモデルの数を色々試し、100%に近づけたい
# 作成するモデルの数を増やすと実行に時間がかかる…
model4 = AdaBoostClassifier(n_estimators=300,random_state=0,base_estimator= base_model)
model4.fit(x_train,y_train)
print(model4.score(x_test,y_test))

model4 = AdaBoostClassifier(n_estimators=750,random_state=0,base_estimator= base_model)
model4.fit(x_train,y_train)
print(model4.score(x_test,y_test))

model4 = AdaBoostClassifier(n_estimators=1000,random_state=0,base_estimator= base_model)
model4.fit(x_train,y_train)
print(model4.score(x_test,y_test))

0.9861111111111112
0.9888888888888889
0.9916666666666667


In [ ]:
# 最終結果一覧 正解率の降順
# 1位：アダブースト99.16%
# 2位：ランダムフォレスト97.22%
# 3位：ロジスティック回帰96.94%

In [ ]:
# アダブーストで作成したモデルを保存する
import pickle
with open('digits.pkl','wb')as f :
  pickle.dump(model4,f)

In [ ]:
# 未知のデータでの予測ができていない